In [91]:
import json
import sys
from rich import print as rprint

from pathlib import Path
from itertools import islice

nb_dir = Path.cwd()

project_root = nb_dir.parent.parent

sys.path.insert(0, str(project_root))


In [92]:
people_records_prepped_file = Path(project_root / "data/people/people_records_prepped.json")
books2people_prepped_file = Path(project_root / "data/people/books2people_prepped.json")
folder_matched = Path(project_root / "data/matched")

validation_log_file = Path(project_root / "data/logs/validation_failed_log.json")

In [93]:
people_records = []
books2people_records = []

with open(people_records_prepped_file, "r") as f:
    people_records = json.load(f)

with open(books2people_prepped_file, "r") as f:
    books2people_records = json.load(f)

books2people_dict = {entry["composite_id"]: entry for entry in books2people_records}
people_dict = {person["unified_id"]: person for person in people_records}


In [ ]:
processed_file_count = 0
validated_entries_total = 0
failed_entries_total = 0

failed_entries = {}

for file in folder_matched.iterdir():
    validated_entries = {}
    validated_count = 0

    if not file.exists():
        raise FileNotFoundError(f"{file} doesn't exist!")

    with open(file, "r") as f:
        books = json.load(f)
        #rprint(books)
        processed_file_count += 1

    for composite_id, book in books.items():
        authors = book["parsed_entry"].get("authors") or []
        editors = book["parsed_entry"].get("editors") or []
        contributors = book["parsed_entry"].get("contributors") or []
        translator = book["parsed_entry"]["translator"]
        is_translation = book["parsed_entry"]["is_translation"]

        authors_exp = len(authors)
        editors_exp = len(editors)
        contributors_exp = len(contributors)
        translator_exp = 1 if translator else 0
        expected_total = authors_exp + editors_exp + contributors_exp + translator_exp
        # print(expected_total)


        # validation logic goes here

        # if validated:
        #     validated_count += 1
